# Named Entity Recognition (NER) Using BERT+CRF
    

## Load data

In [1]:
!pip install -qq datasets
!pip install -qq seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from datasets import load_dataset

dataset = load_dataset("conll2003")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

## BERT-CRF

## Data Loading and Processing

In [4]:
from typing import Dict, List, Optional
from pathlib import Path
import logging

class NERConfig:
    def __init__(
        self,
        label_types: List[str],
        id2label: Dict[int, str],
        max_len: int = 128,
        data_path: Optional[str | Path] = None,
        base_model_path: Optional[str] = None,
        models_path: Optional[str | Path] = None,
    ):
        if isinstance(data_path, str):
            data_path = Path(data_path)
        self.DATA_PATH = data_path
        if isinstance(models_path, str):
            self.MODEL_PATH = Path(models_path)
        self.BASE_MODEL_PATH = base_model_path or "bert-base-cased"
        self.LABEL_TYPES = label_types
        self.ID2LABEL = id2label
        self.LABEL2ID = {label: id for id, label in id2label.items()}
        self.MAX_LEN = max_len
        self.TRAIN_BATCH_SIZE = 64
        self.VALID_BATCH_SIZE = 32
        self.EPOCHS = 15
        self.OUT_DIM = 768  # bert-base-cased: 768, bert-large-cased: 1024
        self.MODEL_PATH = "./bert+crf_model.pt"
        self.TOKENIZER = transformers.BertTokenizer.from_pretrained(
            self.BASE_MODEL_PATH, do_lower_case=False
        )
        self.logger = logging.getLogger(self.__class__.__name__)

In [5]:
import torch
import transformers

LABEL_TYPES = dataset["train"].features.flatten()["ner_tags"].feature.names
ID2LABEL = {id: label for id, label in enumerate(LABEL_TYPES)}
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    "bert-large-cased", do_lower_case=False
)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

In [6]:
ID2LABEL

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [7]:
from typing import List, Dict, Optional

import torch
import numpy as np

# from ner_system.config import NERConfig

ner_config = NERConfig(
    label_types=LABEL_TYPES,
    id2label=ID2LABEL,
    max_len=128,
    base_model_path="bert-base-cased",
)


class CoNLL2003Document:
    """
    Class to represent a single CoNLL2003 document in its base form.

    Supports target labels as integers (if coming from `datasets` library)
    or strings (if coming from the "traditional" CoNLL2003 format).
    """

    def __init__(
        self,
        unique_id: int,
        tokens: List[str],
        ner_tags: List[str] | List[int],
    ):
        self.unique_id = unique_id
        self.tokens = tokens
        if isinstance(ner_tags[0], int):
            self.ner_tags_str = [ner_config.ID2LABEL[tag] for tag in ner_tags]
            self.ner_tags_int = ner_tags
        elif isinstance(ner_tags[0], str):
            self.ner_tags_int = [ner_config.LABEL2ID[tag] for tag in ner_tags]
            self.ner_tags_str = ner_tags

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        _token = self.tokens[idx]
        _tag = self.ner_tags_str[idx]
        return (_token, _tag)

    def to_dict(self):
        return {
            "unique_id": self.unique_id,
            "tokens": self.tokens,
            "ner_tags_str": self.ner_tags_str,
            "ner_tags_int": self.ner_tags_int,
        }


class CoNLL2003Features:
    """
    Class to represent a single CoNLL2003 document in its featureized form.

    A result from the `_convert_to_features` method of `NERDataset`.
    """

    def __init__(
        self,
        input_ids: List[int],
        input_mask: List[int],
        crf_mask: List[int],  # CRF() requires specific mask to avoid: ValueError: mask of the first timestep must all be on
        token_type_ids: List[int],
        predict_mask: List[int],
        label_ids: List[int],
    ):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.crf_mask = crf_mask
        self.token_type_ids = token_type_ids
        self.predict_mask = predict_mask
        self.label_ids = label_ids

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
class NERDataset:
    def __init__(self, data: List[CoNLL2003Document]):
        self.data = data
        # Set IDs of special tokens
        self.CLS = 101
        self.SEP = 102
        self.PAD = 0

    def __len__(self):
        return len(self.data)

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, value):
        self._data = value

    def __getitem__(self, idx):
        _doc = self.data[idx]
        _feat = self._convert_to_features(_doc)
        return {
            "input_ids": torch.LongTensor(_feat.input_ids),
            "input_mask": torch.ByteTensor(_feat.input_mask),
            "crf_mask": torch.ByteTensor(_feat.crf_mask),
            "token_type_ids": torch.LongTensor(_feat.token_type_ids),
            "predict_mask": torch.ByteTensor(_feat.predict_mask),
            "label_ids": torch.LongTensor(_feat.label_ids)
        }

    def _convert_to_features(self, document: CoNLL2003Document) -> CoNLL2003Features:
        # Initializes sequence
        input_ids = []
        label_ids = document.ner_tags_int
        predict_mask = []

        for i, token in enumerate(document.tokens):
            sub_tokens = ner_config.TOKENIZER.tokenize(token)
            predict_mask.extend([1] + [0] * (len(sub_tokens) - 1))
            input_ids.extend(sub_tokens)

        input_ids = ner_config.TOKENIZER.convert_tokens_to_ids(input_ids)
        input_ids = input_ids[:ner_config.MAX_LEN - 2]
        predict_mask = predict_mask[:ner_config.MAX_LEN - 2]
        label_ids = label_ids[:ner_config.MAX_LEN - 2]

        input_mask = [1] * len(input_ids)
        crf_mask = [1] * len(input_ids)
        token_type_ids = [0] * ner_config.MAX_LEN

        input_ids = [self.CLS] + input_ids + [self.SEP]
        label_ids = [0] + label_ids + [0]
        input_mask = [0] + input_mask + [0]
        crf_mask = [1] + crf_mask + [1]
        predict_mask = [1] + predict_mask + [1]

        # print(f"crf_mask: {crf_mask}")
        # print(f"crf_mask_len: {len(crf_mask)}")

        padding_len = ner_config.MAX_LEN - len(input_ids)

        input_ids += [self.PAD] * padding_len
        input_mask += [self.PAD] * padding_len
        crf_mask += [self.PAD] * padding_len
        predict_mask += [self.PAD] * padding_len

        # print(f"crf_mask_padded: {crf_mask}")

        label_padding = ner_config.MAX_LEN - len(label_ids)
        label_ids += [self.PAD] * label_padding

        # print(len(input_ids))
        # print(len(input_mask))
        # print(len(crf_mask))
        # print(len(token_type_ids))
        # print(len(predict_mask))
        # print(len(label_ids))

        # Hard tests
        assert len(input_ids) == len(input_mask) == len(crf_mask) == len(token_type_ids) == len(predict_mask) == len(label_ids) == ner_config.MAX_LEN

        return CoNLL2003Features(
            input_ids, input_mask, crf_mask, token_type_ids, predict_mask, label_ids
        )


    @classmethod
    def pad(cls, batch: List[Dict[str, torch.Tensor]]):
        _seq_lens = [len(sample[0]) for sample in batch]
        _maxlen = np.array(_seq_lens).max()

        def _f(x, seqlen):
            return [sample[x] + [0] * (seqlen - len(sample[x])) for sample in batch]

        input_ids_list = torch.LongTensor(_f(0, ner_config.MAX_LEN))
        input_mask_list = torch.LongTensor(_f(1, ner_config.MAX_LEN))
        crf_mask_list = torch.ByteTensor(_f(2, ner_config.MAX_LEN))
        token_type_ids_list = torch.LongTensor(_f(3, ner_config.MAX_LEN))
        predict_mask_list = torch.ByteTensor(_f(4, ner_config.MAX_LEN))
        label_ids_list = torch.LongTensor(_f(5, ner_config.MAX_LEN))

        # print(f"crf_mask_padded: {crf_mask_list}")
        # print(f"crf_mask_padded_shape: {crf_mask_list.shape}")
        # print(f"crf_mask[0]: {crf_mask_list[0]}")

        assert crf_mask_list[:, 0].all()

        return {
            "input_ids": input_ids_list,
            "input_mask": input_mask_list,
            "crf_mask": crf_mask_list,
            "token_type_ids": token_type_ids_list,
            "predict_mask": predict_mask_list,
            "label_ids": label_ids_list
        }

In [9]:
def convert_to_conll2003_document(
    tokens: List[str],
    ner_tags: List[int],
    unique_id: int,
) -> CoNLL2003Document:
    """
    Converts a single document from the CoNLL2003 dataset to a CoNLL2003Document object.
    """
    return CoNLL2003Document(tokens=tokens, ner_tags=ner_tags, unique_id=unique_id)


# _dummy = dataset["train"][:10]
# dummy = [
#     convert_to_conll2003_document(
#         tokens=toks,
#         ner_tags=tags,
#         unique_id=id,
#     )
#     for toks, tags, id in zip(_dummy["tokens"], _dummy["ner_tags"], _dummy["id"])
# ]
# a = NERDataset(dummy)

In [10]:
from datasets import Dataset


def prepare_NER_data(dataset: Dataset, split: str = None):
    if not split:
        ValueError("At least one of `train`, `validation` or `test` must be provided.")

    if split not in ["train", "validation", "test"]:
        ValueError("`split` must be one of `train`, `validation` or `test`.")

    _data = dataset[split]
    _docs = [
        convert_to_conll2003_document(
            tokens=toks,
            ner_tags=tags,
            unique_id=id,
        )
        for toks, tags, id in zip(_data["tokens"], _data["ner_tags"], _data["id"])
    ]

    return NERDataset(_docs)

In [11]:
from torch.utils.data import DataLoader

train_data = prepare_NER_data(dataset, split="train")
val_data = prepare_NER_data(dataset, split="validation")
test_data = prepare_NER_data(dataset, split="test")

train_loader = DataLoader(
    dataset=train_data,
    batch_size=ner_config.TRAIN_BATCH_SIZE,
    shuffle=True,
    # collate_fn=NERDataset.pad
)
valid_loader = DataLoader(
    dataset=val_data,
    batch_size=ner_config.VALID_BATCH_SIZE,
    shuffle=True,
    # collate_fn=NERDataset.pad
)
test_loader = DataLoader(
    dataset=test_data,
    batch_size=ner_config.VALID_BATCH_SIZE,
    shuffle=False,
    # collate_fn=NERDataset.pad
)

In [12]:
for data in train_loader:
    batch = {k: v.to(DEVICE) for k, v in data.items()}
    print(batch)
    break

{'input_ids': tensor([[  101, 16944,  1121,  ...,     0,     0,     0],
        [  101,  1244,  1138,  ...,     0,     0,     0],
        [  101,   149, 11414,  ...,     0,     0,     0],
        ...,
        [  101, 18684,  1116,  ...,     0,     0,     0],
        [  101, 18149,  1174,  ...,     0,     0,     0],
        [  101, 25808, 21564,  ...,     0,     0,     0]], device='cuda:0'), 'input_mask': tensor([[0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8), 'crf_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0, 

In [13]:
batch.keys()

dict_keys(['input_ids', 'input_mask', 'crf_mask', 'token_type_ids', 'predict_mask', 'label_ids'])

In [14]:
batch["label_ids"][20]

tensor([0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [15]:
batch["input_mask"][20]

tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0', dtype=torch.uint8)

In [16]:
batch["crf_mask"][20]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0', dtype=torch.uint8)

### Model Training

In [17]:
import torch.nn as nn

# NOTE: Forked from https://pytorch-crf.readthedocs.io/en/stable/_modules/torchcrf.html#CRF

class CRF(nn.Module):
    """Conditional random field.

    This module implements a conditional random field [LMP01]_. The forward computation
    of this class computes the log likelihood of the given sequence of tags and
    emission score tensor. This class also has `~CRF.decode` method which finds
    the best tag sequence given an emission score tensor using `Viterbi algorithm`_.

    Args:
        num_tags: Number of tags.
        batch_first: Whether the first dimension corresponds to the size of a minibatch.

    Attributes:
        start_transitions (`~torch.nn.Parameter`): Start transition score tensor of size
            ``(num_tags,)``.
        end_transitions (`~torch.nn.Parameter`): End transition score tensor of size
            ``(num_tags,)``.
        transitions (`~torch.nn.Parameter`): Transition score tensor of size
            ``(num_tags, num_tags)``.


    .. [LMP01] Lafferty, J., McCallum, A., Pereira, F. (2001).
       "Conditional random fields: Probabilistic models for segmenting and
       labeling sequence data". *Proc. 18th International Conf. on Machine
       Learning*. Morgan Kaufmann. pp. 282–289.

    .. _Viterbi algorithm: https://en.wikipedia.org/wiki/Viterbi_algorithm
    """

    def __init__(self, num_tags: int, batch_first: bool = False) -> None:
        if num_tags <= 0:
            raise ValueError(f'invalid number of tags: {num_tags}')
        super().__init__()
        self.num_tags = num_tags
        self.batch_first = batch_first
        self.start_transitions = nn.Parameter(torch.empty(num_tags))
        self.end_transitions = nn.Parameter(torch.empty(num_tags))
        self.transitions = nn.Parameter(torch.empty(num_tags, num_tags))

        self.reset_parameters()

    def reset_parameters(self) -> None:
        """Initialize the transition parameters.

        The parameters will be initialized randomly from a uniform distribution
        between -0.1 and 0.1.
        """
        nn.init.uniform_(self.start_transitions, -0.1, 0.1)
        nn.init.uniform_(self.end_transitions, -0.1, 0.1)
        nn.init.uniform_(self.transitions, -0.1, 0.1)


    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(num_tags={self.num_tags})'

    def forward(
            self,
            emissions: torch.Tensor,
            tags: torch.LongTensor,
            mask: Optional[torch.ByteTensor] = None,
            reduction: str = 'sum',
    ) -> torch.Tensor:
        """Compute the conditional log likelihood of a sequence of tags given emission scores.

        Args:
            emissions (`~torch.Tensor`): Emission score tensor of size
                ``(seq_length, batch_size, num_tags)`` if ``batch_first`` is ``False``,
                ``(batch_size, seq_length, num_tags)`` otherwise.
            tags (`~torch.LongTensor`): Sequence of tags tensor of size
                ``(seq_length, batch_size)`` if ``batch_first`` is ``False``,
                ``(batch_size, seq_length)`` otherwise.
            mask (`~torch.ByteTensor`): Mask tensor of size ``(seq_length, batch_size)``
                if ``batch_first`` is ``False``, ``(batch_size, seq_length)`` otherwise.
            reduction: Specifies  the reduction to apply to the output:
                ``none|sum|mean|token_mean``. ``none``: no reduction will be applied.
                ``sum``: the output will be summed over batches. ``mean``: the output will be
                averaged over batches. ``token_mean``: the output will be averaged over tokens.

        Returns:
            `~torch.Tensor`: The log likelihood. This will have size ``(batch_size,)`` if
            reduction is ``none``, ``()`` otherwise.
        """
        self._validate(emissions, tags=tags, mask=mask)
        if reduction not in ('none', 'sum', 'mean', 'token_mean'):
            raise ValueError(f'invalid reduction: {reduction}')
        if mask is None:
            mask = torch.ones_like(tags, dtype=torch.uint8)

        if self.batch_first:
            emissions = emissions.transpose(0, 1)
            tags = tags.transpose(0, 1)
            mask = mask.transpose(0, 1)

        # shape: (batch_size,)
        numerator = self._compute_score(emissions, tags, mask)
        # shape: (batch_size,)
        denominator = self._compute_normalizer(emissions, mask)
        # shape: (batch_size,)
        llh = numerator - denominator

        if reduction == 'none':
            return llh
        if reduction == 'sum':
            return llh.sum()
        if reduction == 'mean':
            return llh.mean()
        assert reduction == 'token_mean'
        return llh.sum() / mask.float().sum()

    def decode(self, emissions: torch.Tensor,
               mask: Optional[torch.ByteTensor] = None) -> List[List[int]]:
        """Find the most likely tag sequence using Viterbi algorithm.

        Args:
            emissions (`~torch.Tensor`): Emission score tensor of size
                ``(seq_length, batch_size, num_tags)`` if ``batch_first`` is ``False``,
                ``(batch_size, seq_length, num_tags)`` otherwise.
            mask (`~torch.ByteTensor`): Mask tensor of size ``(seq_length, batch_size)``
                if ``batch_first`` is ``False``, ``(batch_size, seq_length)`` otherwise.

        Returns:
            List of list containing the best tag sequence for each batch.
        """
        self._validate(emissions, mask=mask)
        if mask is None:
            mask = emissions.new_ones(emissions.shape[:2], dtype=torch.uint8)

        if self.batch_first:
            emissions = emissions.transpose(0, 1)
            mask = mask.transpose(0, 1)

        return self._viterbi_decode(emissions, mask)


    def _validate(
            self,
            emissions: torch.Tensor,
            tags: Optional[torch.LongTensor] = None,
            mask: Optional[torch.ByteTensor] = None) -> None:
        if emissions.dim() != 3:
            raise ValueError(f'emissions must have dimension of 3, got {emissions.dim()}')
        if emissions.size(2) != self.num_tags:
            raise ValueError(
                f'expected last dimension of emissions is {self.num_tags}, '
                f'got {emissions.size(2)}')

        if tags is not None:
            if emissions.shape[:2] != tags.shape:
                raise ValueError(
                    'the first two dimensions of emissions and tags must match, '
                    f'got {tuple(emissions.shape[:2])} and {tuple(tags.shape)}')

        if mask is not None:
            if emissions.shape[:2] != mask.shape:
                raise ValueError(
                    'the first two dimensions of emissions and mask must match, '
                    f'got {tuple(emissions.shape[:2])} and {tuple(mask.shape)}')
            no_empty_seq = not self.batch_first and mask[0].all()
            no_empty_seq_bf = self.batch_first and mask[:, 0].all()
            if not no_empty_seq and not no_empty_seq_bf:
                raise ValueError('mask of the first timestep must all be on')

    def _compute_score(
            self, emissions: torch.Tensor, tags: torch.LongTensor,
            mask: torch.ByteTensor) -> torch.Tensor:
        # emissions: (seq_length, batch_size, num_tags)
        # tags: (seq_length, batch_size)
        # mask: (seq_length, batch_size)
        assert emissions.dim() == 3 and tags.dim() == 2
        assert emissions.shape[:2] == tags.shape
        assert emissions.size(2) == self.num_tags
        assert mask.shape == tags.shape
        assert mask[0].all()

        seq_length, batch_size = tags.shape
        mask = mask.float()

        # Start transition score and first emission
        # shape: (batch_size,)
        # print(tags[0])
        # print(tags.shape)
        score = self.start_transitions[tags[0]]
        score += emissions[0, torch.arange(batch_size), tags[0]]

        for i in range(1, seq_length):
            # Transition score to next tag, only added if next timestep is valid (mask == 1)
            # shape: (batch_size,)
            score += self.transitions[tags[i - 1], tags[i]] * mask[i]

            # Emission score for next tag, only added if next timestep is valid (mask == 1)
            # shape: (batch_size,)
            score += emissions[i, torch.arange(batch_size), tags[i]] * mask[i]

        # End transition score
        # shape: (batch_size,)
        seq_ends = mask.long().sum(dim=0) - 1
        # shape: (batch_size,)
        last_tags = tags[seq_ends, torch.arange(batch_size)]
        # shape: (batch_size,)
        score += self.end_transitions[last_tags]

        return score

    def _compute_normalizer(
            self, emissions: torch.Tensor, mask: torch.ByteTensor) -> torch.Tensor:
        # emissions: (seq_length, batch_size, num_tags)
        # mask: (seq_length, batch_size)
        assert emissions.dim() == 3 and mask.dim() == 2
        assert emissions.shape[:2] == mask.shape
        assert emissions.size(2) == self.num_tags
        assert mask[0].all()

        seq_length = emissions.size(0)

        # Start transition score and first emission; score has size of
        # (batch_size, num_tags) where for each batch, the j-th column stores
        # the score that the first timestep has tag j
        # shape: (batch_size, num_tags)
        score = self.start_transitions + emissions[0]

        for i in range(1, seq_length):
            # Broadcast score for every possible next tag
            # shape: (batch_size, num_tags, 1)
            broadcast_score = score.unsqueeze(2)

            # Broadcast emission score for every possible current tag
            # shape: (batch_size, 1, num_tags)
            broadcast_emissions = emissions[i].unsqueeze(1)

            # Compute the score tensor of size (batch_size, num_tags, num_tags) where
            # for each sample, entry at row i and column j stores the sum of scores of all
            # possible tag sequences so far that end with transitioning from tag i to tag j
            # and emitting
            # shape: (batch_size, num_tags, num_tags)
            next_score = broadcast_score + self.transitions + broadcast_emissions

            # Sum over all possible current tags, but we're in score space, so a sum
            # becomes a log-sum-exp: for each sample, entry i stores the sum of scores of
            # all possible tag sequences so far, that end in tag i
            # shape: (batch_size, num_tags)
            next_score = torch.logsumexp(next_score, dim=1)

            # Set score to the next score if this timestep is valid (mask == 1)
            # shape: (batch_size, num_tags)
            score = torch.where(mask[i].unsqueeze(1), next_score, score)

        # End transition score
        # shape: (batch_size, num_tags)
        score += self.end_transitions

        # Sum (log-sum-exp) over all possible tags
        # shape: (batch_size,)
        return torch.logsumexp(score, dim=1)

    def _viterbi_decode(self, emissions: torch.FloatTensor,
                        mask: torch.ByteTensor) -> List[List[int]]:
        # emissions: (seq_length, batch_size, num_tags)
        # mask: (seq_length, batch_size)
        assert emissions.dim() == 3 and mask.dim() == 2
        assert emissions.shape[:2] == mask.shape
        assert emissions.size(2) == self.num_tags
        assert mask[0].all()

        seq_length, batch_size = mask.shape

        # Start transition and first emission
        # shape: (batch_size, num_tags)
        score = self.start_transitions + emissions[0]
        history = []

        # score is a tensor of size (batch_size, num_tags) where for every batch,
        # value at column j stores the score of the best tag sequence so far that ends
        # with tag j
        # history saves where the best tags candidate transitioned from; this is used
        # when we trace back the best tag sequence

        # Viterbi algorithm recursive case: we compute the score of the best tag sequence
        # for every possible next tag
        for i in range(1, seq_length):
            # Broadcast viterbi score for every possible next tag
            # shape: (batch_size, num_tags, 1)
            broadcast_score = score.unsqueeze(2)

            # Broadcast emission score for every possible current tag
            # shape: (batch_size, 1, num_tags)
            broadcast_emission = emissions[i].unsqueeze(1)

            # Compute the score tensor of size (batch_size, num_tags, num_tags) where
            # for each sample, entry at row i and column j stores the score of the best
            # tag sequence so far that ends with transitioning from tag i to tag j and emitting
            # shape: (batch_size, num_tags, num_tags)
            next_score = broadcast_score + self.transitions + broadcast_emission

            # Find the maximum score over all possible current tag
            # shape: (batch_size, num_tags)
            next_score, indices = next_score.max(dim=1)

            # Set score to the next score if this timestep is valid (mask == 1)
            # and save the index that produces the next score
            # shape: (batch_size, num_tags)
            score = torch.where(mask[i].unsqueeze(1), next_score, score)
            history.append(indices)

        # End transition score
        # shape: (batch_size, num_tags)
        score += self.end_transitions

        # Now, compute the best path for each sample

        # shape: (batch_size,)
        seq_ends = mask.long().sum(dim=0) - 1
        best_tags_list = []

        for idx in range(batch_size):
            # Find the tag which maximizes the score at the last timestep; this is our best tag
            # for the last timestep
            _, best_last_tag = score[idx].max(dim=0)
            best_tags = [best_last_tag.item()]

            # We trace back where the best last tag comes from, append that to our best tag
            # sequence, and trace it back again, and so on
            for hist in reversed(history[:seq_ends[idx]]):
                best_last_tag = hist[idx][best_tags[-1]]
                best_tags.append(best_last_tag.item())

            # Reverse the order because we start from the last timestep
            # assert len(best_tags) == seq_length, "length of best_tags: {} != seq_length: {}".format(len(best_tags), seq_length)
            best_tags.reverse()
            best_tags_list.append(best_tags)

        return best_tags_list

In [18]:
import logging

import numpy as np

import torch
import torch.nn as nn
from torch.optim import AdamW, Optimizer
from torch.optim.lr_scheduler import LRScheduler
#from torchcrf import CRF

from transformers import get_linear_schedule_with_warmup

from seqeval.metrics import classification_report, accuracy_score, f1_score

from tqdm import tqdm


def filter_sequence_output(sequence_of_tags, input_mask, predict_mask):
    batch_size = len(sequence_of_tags)
    max_len = ner_config.MAX_LEN
    current_len = len(sequence_of_tags[0])

    filtered_tags = torch.zeros(
        batch_size,
        max_len,
        dtype=torch.long,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )
    filtered_input_mask = torch.zeros(
        batch_size,
        max_len,
        dtype=torch.long,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )

    # print(f"input_mask (inside): {input_mask}")
    # print(f"input_mask (inside): {input_mask.shape}")

    c = 0
    try:
        for i in range(batch_size):
            jj = -1
            for j in range(current_len):
                if predict_mask[i][j].item() == 1:
                    jj += 1
                    # print(f"input_mask[i][j]: {input_mask[i][j]}")
                    # print(f"sequence_of_tags[i][j]: {sequence_of_tags[i][j]}")
                    filtered_tags[i][jj] = sequence_of_tags[i][j] * input_mask[i][j]
                    filtered_input_mask[i][jj] = input_mask[i][j]
                    c += 1
    except Exception as e:
        print(f"c: {c}")
        print(f"i: {i}, j: {j}, jj: {jj}")
        print(f"len sequence_of_tags: {len(sequence_of_tags)}")
        print(f"len sequence_of_tags[i]: {len(sequence_of_tags[i])}")
        print(f"predict_mask: {predict_mask.shape}")
        print(f"filtered_tags: {filtered_tags.shape}")
        print(f"filtered_input_mask: {filtered_input_mask.shape}")
        raise(e)

    return filtered_tags, filtered_input_mask


class BERTCRFModel(nn.Module):
    def __init__(
        self,
        num_tags: int,
        out_dim: int,
        batch_first: bool = True,
        dropout: float = None,
    ):
        super(BERTCRFModel, self).__init__()
        self.num_tags = num_tags
        self.out_dim = out_dim
        if dropout:
            self.dropout = nn.Dropout(dropout)
        self.bert = transformers.BertModel.from_pretrained(
            ner_config.BASE_MODEL_PATH, return_dict=False
        )
        self.linear = nn.Linear(self.out_dim, self.num_tags)
        # self.linear.bias.data[0] = 6
        self.crf = CRF(num_tags, batch_first=batch_first)
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.info(f"Initialized BERTCRFModel with the following params: {self.__dict__}")
        print(f"Initialized BERTCRFModel with the following params: {self.__dict__}")

    def forward(
        self, input_ids, input_mask, crf_mask, token_type_ids, predict_mask, label_ids
    ):
        o1, _ = self.bert(
            input_ids=input_ids,
            attention_mask=input_mask,
            token_type_ids=token_type_ids,
        )
        if self.dropout:
            o1 = self.dropout(o1)
        # Take encoded output and compute its emission matrices
        # print(o1.shape)
        # print(model.linear.weight.shape)
        emissions = self.linear(o1)

        # Compute the negative log-likelihood for the CRF
        # print(f"crf_mask: {crf_mask.shape}")
        # print(f"crf_mask[0]: {crf_mask[0]}")
        # print(f"crf_mask[0][0]: {crf_mask[0][0]}")
        # print(f"crf_mask[0][0].item(): {crf_mask[0][0].item()}")

        # print(f"input_mask: {input_mask.shape}")
        # print(f"input_mask[0]: {input_mask[0]}")
        # print(f"input_mask[0][0]: {input_mask[0][0]}")
        # print(f"input_mask[0][0].item(): {input_mask[0][0].item()}")

        # crf_mask[:, 0].fill_(1)  # Mitigates: ValueError: mask of the first timestep must all be on

        # # print(f"crf_mask[:, 0]: {crf_mask[:, 0]}")

        # Hard check to make sure that the CRF model runs (See docs)
        assert crf_mask[:, 0].all()

        # print(f"inputs_ids: {input_ids[20]}")
        # print(f"input_mask: {input_mask[20]}")
        # print(f"crf_mask: {crf_mask[20]}")
        # print(f"predict_mask: {predict_mask[20]}")
        # print(f"label_ids: {label_ids[20]}")
        # print(f"emissions: {emissions[20]}")

        log_likelihood = self.crf(
            emissions=emissions,
            tags=label_ids,
            mask=crf_mask.bool(),
            reduction="mean"
        )

        # Decode the most likely sequence of tags
        sequence_of_tags = self.crf.decode(emissions, mask=crf_mask.bool())

        # print(f"sequence_of_tags: {sequence_of_tags}")

        # Retrieve the target tag given the input and predicted masks
        pred_tags, filtered_mask = filter_sequence_output(
            sequence_of_tags, input_mask, predict_mask
        )
        loss = -1 * log_likelihood

        return pred_tags, label_ids, filtered_mask, loss

In [19]:
def train(
    dataloader: DataLoader,
    model: BERTCRFModel,
    optimizer: Optimizer,
    device: torch.device,
    scheduler: Optional[LRScheduler] = None,
):
    model.train()

    final_loss = 0
    y_gold = []
    y_pred = []

    for data in tqdm(dataloader, total=len(dataloader)):
        batch = {k: v.to(DEVICE) for k, v in data.items()}
        optimizer.zero_grad()
        # print(batch.keys())

        pred_tags, gold_tags, mask, loss = model(**batch)
        # print(f"pred_tags: {pred_tags[20]}")
        # print(f"gold_tags: {gold_tags[20]}")
        # print(f"mask: {mask[20]}")
        # print(f"mask: {mask.shape}")
        # print(f"loss: {loss}")

        mapped_gold = [
            ner_config.ID2LABEL[item]
            for item in torch.flatten(mask * gold_tags).detach().cpu().tolist()
        ]
        mapped_pred = [
            ner_config.ID2LABEL[item]
            for item in torch.flatten(mask * pred_tags).detach().cpu().tolist()
        ]
        # print(f"mapped_gold: {mapped_gold}")
        # print(f"mapped_pred: {mapped_pred}")
        y_gold.append(mapped_gold)
        y_pred.append(mapped_pred)

        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()

    # print(f"y_gold: {y_gold}")
    # print(f"y_pred: {y_pred}")

    return final_loss / len(dataloader), f1_score(y_gold, y_pred)


def evaluate(dataloader: DataLoader, model: BERTCRFModel, device: torch.device):
    model.eval()

    final_loss = 0
    y_gold = []
    y_pred = []

    with torch.no_grad():
        for data in tqdm(dataloader, total=len(dataloader)):
            batch = {k: v.to(DEVICE) for k, v in data.items()}

            pred_tags, gold_tags, mask, loss = model(**batch)
            # print(f"pred_tags: {pred_tags[20]}")
            # print(f"gold_tags: {gold_tags[20]}")
            # print(f"mask: {mask[20]}")
            mapped_target = [
                ner_config.ID2LABEL[item]
                for item in torch.flatten(mask * gold_tags).detach().cpu().tolist()
            ]
            mapped_output = [
                ner_config.ID2LABEL[item]
                for item in torch.flatten(mask * pred_tags).detach().cpu().tolist()
            ]
            y_gold.append(mapped_target)
            y_pred.append(mapped_output)

            final_loss += loss.item()

    return final_loss / len(dataloader), f1_score(y_gold, y_pred)

In [20]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BERTCRFModel(
    num_tags=len(ner_config.LABEL_TYPES),
    out_dim=ner_config.OUT_DIM,
    batch_first=True,
    dropout=0.1,
).to(DEVICE)

optim_parameters = [
    {"params": model.bert.parameters(), "lr": 5e-5},
    {"params": model.linear.parameters(), "lr": 1e-3},
    {"params": model.crf.parameters(), "lr": 1e-3},
]

optimizer = AdamW(optim_parameters, weight_decay=0.01)

_num_train_steps = (
    dataset["train"].num_rows / ner_config.TRAIN_BATCH_SIZE * ner_config.EPOCHS
)
num_train_steps = int(_num_train_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_train_steps*0.1, num_training_steps=num_train_steps
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Initialized BERTCRFModel with the following params: {'training': True, '_parameters': OrderedDict(), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_pre_hooks': OrderedDict(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_hooks_with_kwargs': OrderedDict(), '_forward_hooks_always_called': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_forward_pre_hooks_with_kwargs': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_post_hooks': OrderedDict(), '_modules': OrderedDict([('dropout', Dropout(p=0.1, inplace=False)), ('bert', BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (d

In [21]:
class EarlyStopping:
    def __init__(self, tolerance=3, min_delta=10):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:
                self.early_stop = True

In [22]:
import os

best_loss = np.inf

early_stopping = EarlyStopping(tolerance=3, min_delta=1)

for epoch in range(ner_config.EPOCHS):
    train_loss, train_f1 = train(
        dataloader=train_loader,
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        device=DEVICE
      )
    val_loss, val_f1 = evaluate(
        dataloader=valid_loader,
        model=model,
        device=DEVICE
      )
    print(f"Train loss = {train_loss} Valid loss = {val_loss} ")
    print(f"Train f1_score = {train_f1} Valid f1_score = {val_f1} ")
    if val_loss < best_loss:
        torch.save(model.state_dict(), ner_config.MODEL_PATH)
        best_loss = val_loss

    # early stopping
    early_stopping(train_loss, val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping activated! Epoch: {epoch}")
        break

    torch.cuda.empty_cache()

100%|██████████| 102/102 [00:17<00:00,  5.96it/s]


Train loss = 13.79386907707561 Valid loss = 7.215043610217524 
Train f1_score = 0.05670061099796334 Valid f1_score = 0.3430709454597784 


100%|██████████| 102/102 [00:17<00:00,  5.97it/s]


Train loss = 5.010112796046517 Valid loss = 3.95094610896765 
Train f1_score = 0.43313538798394324 Valid f1_score = 0.5219087244831622 


100%|██████████| 102/102 [00:16<00:00,  6.01it/s]


...

Train loss = 1.1853196103464474 Valid loss = 2.263224331187267 
Train f1_score = 0.48640563132424114 Valid f1_score = 0.459890610756609 


100%|██████████| 102/102 [00:16<00:00,  6.01it/s]


Train loss = 0.9735116239298474 Valid loss = 2.156189632182028 
Train f1_score = 0.49494437319175594 Valid f1_score = 0.4720614294152392 


100%|██████████| 102/102 [00:16<00:00,  6.05it/s]


Train loss = 0.79375664930452 Valid loss = 2.010067455324472 
Train f1_score = 0.48749381242174533 Valid f1_score = 0.48318562583464847 
Early stopping activated! Epoch: 7


## Evaluation

In [23]:
def evaluate(
    dataloader: DataLoader,
    model: BERTCRFModel,
    device: torch.device,
  ):
    model.eval()

    final_loss = 0
    y_gold = []
    y_pred = []

    with torch.no_grad():
        for data in tqdm(dataloader, total=len(dataloader)):
            batch = {k: v.to(DEVICE) for k, v in data.items()}

            pred_tags, gold_tags, mask, loss = model(**batch)

            mapped_target = [
                ner_config.ID2LABEL[item]
                for item in torch.flatten(mask * gold_tags).detach().cpu().tolist()
            ]
            mapped_output = [
                ner_config.ID2LABEL[item]
                for item in torch.flatten(mask * pred_tags).detach().cpu().tolist()
            ]
            y_gold.append(mapped_target)
            y_pred.append(mapped_output)

            final_loss += loss.item()

    print(classification_report(y_gold, y_pred, digits=2))
    return final_loss/len(dataloader), f1_score(y_gold, y_pred)

In [24]:
test_loss, test_f1 = evaluate(
    dataloader=test_loader,
    model=model,
    device=DEVICE
)
print()
print(f"Test loss = {test_loss} Test f1_score = {test_f1} ")

100%|██████████| 108/108 [00:17<00:00,  6.23it/s]


              precision    recall  f1-score   support

         LOC       0.52      0.40      0.45      1341
        MISC       0.38      0.38      0.38       535
         ORG       0.59      0.61      0.60      1359
         PER       0.34      0.32      0.33      1203

   micro avg       0.48      0.44      0.46      4438
   macro avg       0.46      0.43      0.44      4438
weighted avg       0.48      0.44      0.46      4438


Test loss = 2.891814234632033 Test f1_score = 0.4579132473622509 


## References

* https://www.kaggle.com/code/vanquitu/bert-crf
* https://github.com/Louis-udm/NER-BERT-CRF/blob/master/NER_BERT_CRF.py
* https://github.com/ericput/bert-ner/blob/master/run_ner.py
* https://github.com/FuYanzhe2/Name-Entity-Recognition/blob/master/BERT-BiLSTM-CRF-NER/bert_lstm_ner.py
